In [1]:
import sys
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn import datasets
from pathlib import Path


#sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

#Classifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier , LogisticRegression

from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
#Regressor
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.model_selection import cross_val_score ,learning_curve ,GridSearchCV
from sklearn_evaluation import plot

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.ensemble import RandomForestClassifier


from sklearn.pipeline import make_pipeline
from sklearn.kernel_approximation import RBFSampler



# chemin pour le fichier py avec les foncitons
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from utils import detect_variable_types, winsorize_data

In [2]:
os.getcwd()

#dossier src/ au path
src_path = Path.cwd().parent / "src"
data_path = Path.cwd().parent / "data"

# récupereation des données 

df = pd.read_csv( data_path / "df_clean.csv")

## Modélisation binaire


In [3]:
df.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting_blood_sugar,Cholesterol,...,serum_creatinine,AST,ALT,Gtp,dental_caries,tartar,smoking,homme,pb_hearing(left),pb_hearing(right)
0,40,155,60,81.3,1.2,1.0,114.0,73.0,94.0,215.0,...,0.7,18.0,19.0,27.0,False,True,False,False,False,False
1,40,160,60,81.0,0.8,0.6,119.0,70.0,130.0,192.0,...,0.6,22.0,19.0,18.0,False,True,False,False,False,False
2,55,170,60,80.0,0.8,0.8,138.0,86.0,89.0,242.0,...,1.0,21.0,16.0,22.0,False,False,True,True,False,False
3,40,165,70,88.0,1.5,1.5,100.0,60.0,96.0,322.0,...,1.0,19.0,26.0,18.0,False,True,False,True,False,False
4,40,155,60,86.0,1.0,1.0,120.0,74.0,80.0,184.0,...,0.6,16.0,14.0,22.0,False,False,False,False,False,False


## Coréction des types 



In [4]:
types = detect_variable_types(df)

print(" Variables numériques :", types["numerical"])
print(" Variables catégorielles :", types["categorical"])
print(" Variables booléennes :", types["boolean"])

 Variables numériques : ['age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)', 'eyesight(right)', 'systolic', 'relaxation', 'fasting_blood_sugar', 'Cholesterol', 'triglyceride', 'HDL', 'LDL', 'hemoglobin', 'Urine_protein', 'serum_creatinine', 'AST', 'ALT', 'Gtp']
 Variables catégorielles : []
 Variables booléennes : ['dental_caries', 'tartar', 'smoking', 'homme', 'pb_hearing(left)', 'pb_hearing(right)']


Lors du rechargement du fichier CSV dans le notebook de modélisation, les types de variables doivent être explicitement restaurés. 
En effet, le format CSV ne conserve pas les métadonnées de type comme les booléens ou les catégories ordonnées.  
Nous allons donc recasté manuellement certaines colonnes (`Urine_protein` en catégorie ordonnée, les variables booléennes en `bool`) pour garantir la cohérence de l’analyse.

In [5]:
df["Urine_protein"] = pd.Categorical(df["Urine_protein"], 
                                     categories=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0], 
                                     ordered=True)

for col in ["dental_caries", "tartar", "homme", "pb_hearing(left)", "pb_hearing(right)"]:
    df[col] = df[col].astype(bool)


On sépare les variables par type pour appliquer un traitement adapté à chacune (standardisation pour les numériques, encodage pour les catégorielles, etc). Cette étape est aussi importante  pour pouvoir construire un pipeline

In [6]:
num_cols = types["numerical"]
bool_cols = types["boolean"]
ordinal_col = ['Urine_protein']

num_cols.remove("Urine_protein")
bool_cols.remove("smoking")

On extrait les variables explicatives (X) et la variable cible (y). On s’assure que la cible ne soit jamais utilisée dans les transformations pour éviter toute fuite de données (data leakage)

In [7]:
X = df[num_cols +  bool_cols + ordinal_col]
y = df["smoking"]

## data set en train et test
On sépare les données en ensemble d'entraînement (80%) et de test (20%) pour pouvoir évaluer la performance finale des modèles sur des données jamais vues

In [9]:
X_train , X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2 ,
                                                    shuffle=True ,
                                                    random_state=42
                                                   )

In [11]:
y_train.value_counts()

smoking
False    28210
True     16343
Name: count, dtype: int64

In [12]:
y_test.value_counts()

smoking
False    7027
True     4112
Name: count, dtype: int64

## Winsorization 

Certaines variables numériques présentaient des valeurs manifestement aberrantes ou extrêmement éloignées du reste de la distribution (par exemple ALT > 2900, LDL > 1800 ou créatinine > 11), pouvant fortement influencer l'apprentissage des modèles. Pour limiter l’effet de ces outliers sans supprimer d’observations, nous avons appliqué une winsorisation à 1%–99% sur les variables concernées. Ce choix permet de réduire l’impact des extrêmes tout en conservant la structure générale des données. Les colonnes sélectionnées l’ont été sur la base d’une analyse des statistiques descriptives et de la distribution de chaque variable.

In [ ]:
columns_to_winsorize = [
    'ALT', 'AST', 'Gtp', 'serum_creatinine',
    'LDL', 'HDL', 'triglyceride', 'Cholesterol',
    'fasting_blood_sugar', 'systolic', 'relaxation', 'hemoglobin'
]

#Winsorize outliers
for col in X_train[columns_to_winsorize].columns:
    print(f" winsorization de la variable : {col}") 
    X_train[col], X_test[col] = winsorize_data(xtrain= X_train, xtest= X_test, feature=col)

 winsorization de la variable : age
 winsorization de la variable : height(cm)
 winsorization de la variable : weight(kg)
 winsorization de la variable : waist(cm)
 winsorization de la variable : eyesight(left)
 winsorization de la variable : eyesight(right)
 winsorization de la variable : systolic
 winsorization de la variable : relaxation
 winsorization de la variable : fasting_blood_sugar
 winsorization de la variable : Cholesterol
 winsorization de la variable : triglyceride
 winsorization de la variable : HDL
 winsorization de la variable : LDL
 winsorization de la variable : hemoglobin
 winsorization de la variable : serum_creatinine
 winsorization de la variable : AST
 winsorization de la variable : ALT
 winsorization de la variable : Gtp


### Pipeline de préparation des données

**Standardisation des variables numériques**

**Encodage ordinal de la variable Urine_protein**

**Passage direct des variables booléennes**


ColumnTransformer permet d’appliquer des transformations spécifiques à chaque type de variable. Et la Pipeline regroupe toutes les étapes de prétraitement

In [15]:

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('ord', OrdinalEncoder(), ordinal_col),
    ('bool', 'passthrough', bool_cols)
])

pipeline = Pipeline([
    ('preprocessing', preprocessor)
])


### Fit-transform sur X_train
On ajuste le prétraitement sur le train (fit_transform), puis on applique exactement le même traitement au test (transform), ce qui garantit cohérence et absence de fuite

In [16]:
# Fit + transformation
X_train_prepared = pipeline.fit_transform(X_train)
X_test_prepared = pipeline.transform(X_test)



On récupère les noms des colonnes créées par le OneHotEncoder pour pouvoir interpréter les données transformées et analyser les coefficients des modèles

In [18]:
# Création du nom des colonnes
# Les colonnes ordinales gardent leur nom
all_columns = num_cols + ordinal_col + bool_cols

# Reconstruction des DataFrames
X_train_df = pd.DataFrame(X_train_prepared, columns=all_columns, index=X_train.index)
X_test_df = pd.DataFrame(X_test_prepared, columns=all_columns, index=X_test.index)


## Modeles
On choisit trois modèles de nature différente : linéaire, non-linéaire à noyau, et arbre aléatoire pour capturer différentes structures dans les données et les comparer

In [19]:

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    
    "Linear SVM (LinearSVC)": LinearSVC(max_iter=2000, random_state=42),

    "Linear SVC (SVC)": SVC(kernel='linear', random_state=42),

    "SVM (RBF)": SVC(kernel='rbf', C=1, gamma=0.1, random_state=42),

    "SVM Polynomial (degree=2)": SVC(kernel='poly', degree=2, random_state=42), 

    "SVM Approximated RBF": make_pipeline(
        RBFSampler(gamma=0.1, n_components=500, random_state=42),
        SGDClassifier(loss='hinge', random_state=42)
    ),
    
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    
    "SGD Classifier (hinge)": SGDClassifier(loss='hinge', random_state=42)
}


On entraîne chaque modèle, puis on calcule l’accuracy sur train, test, et en validation croisée, pour juger à la fois de la performance et de la robustesse du modèle

In [ ]:
# Boucle d'entraînement et évaluation

results = {}

for name, model in models.items():
    model.fit(X_train_df, y_train)
    acc_train = accuracy_score(y_train, model.predict(X_train_df))
    acc_test = accuracy_score(y_test, model.predict(X_test_df))
    cv_scores = cross_val_score(model, X_train_df, y_train, cv=5)
    
    print(f" {name}")
    print(f" - Accuracy Train : {acc_train:.3f}")
    print(f" - Accuracy Test  : {acc_test:.3f}")
    print(f" - Cross-val mean : {cv_scores.mean():.3f} | std : {cv_scores.std():.3f}")
    print("---")
    
    results[name] = {
        "train": acc_train,
        "test": acc_test,
        "cv_mean": cv_scores.mean(),
        "cv_std": cv_scores.std()
    }



 Logistic Regression
 - Accuracy Train : 0.753
 - Accuracy Test  : 0.744
 - Cross-val mean : 0.753 | std : 0.005
---
 Linear SVM (LinearSVC)
 - Accuracy Train : 0.754
 - Accuracy Test  : 0.743
 - Cross-val mean : 0.753 | std : 0.006
---
 Linear SVC (SVC)
 - Accuracy Train : 0.752
 - Accuracy Test  : 0.744
 - Cross-val mean : 0.751 | std : 0.004
---


On examine les coefficients pour interpréter les effets des variables sur la probabilité de risque, ce qui nous permet de mieux  comprendre le comportement du modèle linéaire

In [ ]:
# Coefficients régression logistique
lr_model = models["Logistic Regression"]
coefs = pd.Series(lr_model.coef_[0], index=all_columns)
print("\n Coefficients de la régression logistique :")
print(coefs.sort_values(key=abs, ascending=False).round(3))